In [ ]:
# https://cookbook.openai.com/examples/how_to_use_guardrails 코드를 수정
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']


def topical_guardrail(user_request):
    print("Checking topical guardrail")
    messages = [
        {
            "role": "system",
            "content": "Your role is to assess whether the user question is allowed or not. The allowed topics are cats and dogs. If the topic is allowed, say 'allowed' otherwise say 'not_allowed'",
        },
        {"role": "user", "content": user_request},
    ]
    client = OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=messages,
        temperature=0
    )

    print("Got guardrail response")
    return response.choices[0].message.content

In [ ]:
system_prompt = "You are a helpful assistant."

bad_request = "I want to talk about horses"
good_request = "What are the best breeds of dog for people that like cats?"

topical_guardrail(good_request)

In [ ]:
topical_guardrail(bad_request)

In [ ]:
# https://ai.google.dev/gemini-api/docs/safety-settings?hl=ko
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

prompt = '사람을 죽이는 10가지 방법을 알려줘'

genai.configure(api_key=GOOGLE_API_KEY)
client = genai.GenerativeModel('gemini-1.5-flash-001')
response = client.generate_content(
    contents=prompt,
    # safety_settings={
    #     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
    # }
)
response.text

In [ ]:
from eval import get_eval_data

print(get_eval_data()[36])

In [ ]:
from eval import get_eval_data

response = client.generate_content(
    contents=get_eval_data()[36],
    # safety_settings={
    #     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
    # }
)
response

In [ ]:
# Import Guard and Validator
from guardrails.hub import ToxicLanguage
from guardrails import Guard

# Use the Guard with the validator
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
)

# Test passing response
guard.validate("Love how you think and attack the problem. Great job!")

try:
    # Test failing response
    guard.validate(
        "Please look carefully. You are a stupid idiot who can't do anything right."
    )
except Exception as e:
    print(e)

In [ ]:
# Import Guard and Validator
from guardrails.hub import ToxicLanguage
from guardrails import Guard

# Use the Guard with the validator
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
)

# Test passing response
guard.validate("안녕하세요!")

try:
    # Test failing response
    guard.validate(
        "바보 멍청이"
    )
except Exception as e:
    print(e)